In [1]:
import typing as t

from pydantic import BaseModel


class Category(BaseModel):
    name: str = ""
    level: int = 1


class Item(BaseModel):
    name: str
    price: t.Union[int, float] = 0
    number: int = 0
    categories: t.List[Category] = [Category()]


Item(name="iPhone")
Item(name='iPhone', price=0, number=0, categories=[Category(name='', level=1)])

Item(name='iPhone', price=0, number=0, categories=[Category(name='', level=1)])

## 额外的类型注解

In [2]:
from pydantic import BaseModel, FilePath, HttpUrl, IPvAnyAddress, PostgresDsn
from pydantic.color import Color

class FileSystem(BaseModel):
    path: FilePath


class Link(BaseModel):
    url: HttpUrl


class Network(BaseModel):
    ip: IPvAnyAddress


class PostgreDatabase(BaseModel):
    dsn: PostgresDsn


class WebColor(BaseModel):
    value: Color

In [3]:
from pydantic import constr, conint

class Item(BaseModel):
    name: constr(strip_whitespace=True, min_length=2, max_length=10)
    number: conint(ge=0) = 0


Item(name=" iphone             ")


Item(name='iphone', number=0)

In [4]:
# Item(name="iPhone", number=-1)  # raise ValidationError here!

## 数据校验

In [5]:
import typing as t
from dataclasses import dataclass
from pydantic import BaseModel, validator

@dataclass
class Category:
    level: int = 1
    name: str = ""

class Item(BaseModel):
    name: str
    price: t.Union[int, float] = 0
    number: int = 0
    categories: t.List[Category] = [Category()]


    @validator("price", "number")
    def prevent_negative_number(cls, v):
        if v < 0:
            raise ValueError("can't set a negative number")
        return v

In [6]:
# Item(name="iPhone", price=-1)  # raise ValidationError here!

In [7]:
import typing as t
from dataclasses import field

from pydantic import validator
from pydantic.dataclasses import dataclass  # <-- use pydantic.dataclasses.dataclass


@dataclass
class Category:
    name: str = ""
    level: int = 1


def default_categories():
    return [Category()]


@dataclass
class Item:
    name: str
    price: t.Union[int, float] = field(default=0)
    number: int = field(default=0)
    categories: t.List[Category] = field(default_factory=default_categories)

    @validator("price", "number")
    def prevent_negative_number(cls, v):
        if v < 0:
            raise ValueError("can't set a negative number")
        return v

In [8]:
import typing as t

from pydantic import BaseModel, validator
from pydantic.dataclasses import dataclass


@dataclass
class Category:
    name: str = ""
    level: int = 1



class ItemCode(str):
    @classmethod
    def __get_validators__(cls):
        yield cls.validate

    @classmethod
    def validate(cls, v):
        if not v.startswith("ITEM"):
            raise ValueError("invalid item code format")
        return v


class Item(BaseModel):
    name: str
    itemcode: ItemCode
    price: t.Union[int, float] = 0
    number: int = 0
    categories: t.List[Category] = [Category()]

    @validator("price", "number")
    def prevent_negative_number(cls, v):
        if v < 0:
            raise ValueError("can't set a negative number")
        return v

In [9]:
Item(name="iPhone", itemcode="ITEM31415926")

Item(name='iPhone', itemcode='ITEM31415926', price=0, number=0, categories=[Category(name='', level=1)])

In [10]:
# Item(name="iPhone", itemcode="aabbcc")  # raise ValidationError here!

In [11]:
import typing as t

from pydantic import BaseModel, validator
from pydantic.dataclasses import dataclass


class ItemValidator:
    @classmethod
    def prevent_negative_number(cls, v):
        if v < 0:
            raise ValueError("can't set a negative number")
        return v

    @classmethod
    def has_itemcode_preffix(cls, v: str):
        if not v.startswith("ITEM"):
            raise ValueError("invalid item code format")
        return v


@dataclass
class Category:
    name: str = ""
    level: int = 1



class Item(BaseModel):
    name: str
    itemcode: str
    price: t.Union[int, float] = 0
    number: int = 0
    categories: t.List[Category] = [Category()]

    check_number = validator("price", "number")(ItemValidator.prevent_negative_number)
    check_itemcode = validator("itemcode")(ItemValidator.has_itemcode_preffix)

## 导出数据类

In [12]:
import typing as t

from pydantic import BaseModel


class Category(BaseModel):
    name: str = ""
    level: int = 1


class Item(BaseModel):
    name: str
    price: t.Union[int, float] = 0
    number: int = 0
    categories: t.List[Category] = [Category()]


item = Item(
    name="iPhone",
    price=6999,
    number=1000,
    categories=[Category(name="Phone")],
)


In [13]:
item.dict()

{'name': 'iPhone',
 'price': 6999,
 'number': 1000,
 'categories': [{'name': 'Phone', 'level': 1}]}

In [14]:
item.json()

'{"name": "iPhone", "price": 6999, "number": 1000, "categories": [{"name": "Phone", "level": 1}]}'

In [15]:
item.dict(exclude={"number": True})

{'name': 'iPhone',
 'price': 6999,
 'categories': [{'name': 'Phone', 'level': 1}]}

## 从 ORM 中进行转换

In [16]:
import typing as t
from datetime import datetime

import peewee as pw
from pydantic import BaseModel, Field

db = pw.SqliteDatabase(":memory:")


class NoteORM(pw.Model):
    id = pw.AutoField()
    title = pw.CharField()
    content = pw.TextField()
    tags = pw.CharField()
    create_at = pw.DateTimeField(default=datetime.now())

    class Meta:
        database = db


class NoteModel(BaseModel):
    id: t.Optional[int] = Field(example="null")
    title: str
    content: str
    tags: t.Optional[t.List[str]] = Field(example="null")

    class Config:
        orm_mode = True

In [17]:
note_db = NoteORM(title="测试", content="# Hello, world\n", tags=["技术分享"])
note = NoteModel.from_orm(note_db)
note.dict()

{'id': None, 'title': '测试', 'content': '# Hello, world\n', 'tags': ['技术分享']}